In [1]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from glob import glob

In [9]:
# Get tweets
tweets=pd.DataFrame(data=[])
file_list= glob("tweets/*")
for file in tqdm(file_list):
    df_part = pd.read_csv(file)
    df_part['username'] = str(file.split('/')[1][4:-11])
    tweets = tweets.append(df_part)
    
tweets.rename(columns={"0":"id_str", "1":"created_at", "2":"text", "3":"entities", "4":"retweeted"}, inplace=True)

100%|██████████| 1106/1106 [02:30<00:00,  7.36it/s]


In [10]:
tweets.to_csv("data/tweets.csv",index=False)

In [13]:
# Get user lists
cols_to_use = ['username', 'realname', 'gender_', 'age_', 'meslek']
users_age_gender_1 = pd.read_csv('data/labeling/all_groups_notna.csv', usecols=cols_to_use, encoding='utf-8').rename(columns={'age_':'age', 'gender_':'gender'}).drop_duplicates(['username'])
users_age_gender_1 = users_age_gender_1[users_age_gender_1.gender.notna()]

cols_to_use = ['username', 'realname', 'gender', 'age', 'meslek']
users_age_gender_2 = pd.read_csv('data/labeling/basak.csv', usecols=cols_to_use, encoding='utf-8').drop_duplicates(['username'])
users_age_gender_3 = pd.read_csv('data/labeling/basak2.csv', usecols=cols_to_use, encoding='utf-8').drop_duplicates(['username'])
users_age_gender_4 = pd.read_csv('data/labeling/accounts_v4.csv', usecols=cols_to_use, encoding='utf-8').drop_duplicates(['username'])

users_age_gender = users_age_gender_1.append(users_age_gender_2).append(users_age_gender_3).append(users_age_gender_4)
users_age_gender.dropna(subset=['username', 'gender', 'age'], inplace=True)

users_age_gender.age = users_age_gender.age.astype(str).apply(lambda x: x.replace('?','').rstrip().lstrip())
users_age_gender.age = users_age_gender.age.astype(str).apply(lambda x: np.nan if x == '' else x)
users_age_gender.dropna(subset=['username', 'gender', 'age'], inplace=True)

users_age_gender.age = users_age_gender.age.astype(int)
users_age_gender = users_age_gender[users_age_gender.gender != 'u']

print(users_age_gender.shape)
users_age_gender.drop_duplicates(subset=['username'], inplace=True)
print(users_age_gender.shape)
print(users_age_gender.isna().sum())
print(users_age_gender.nunique())

(1317, 5)
(1090, 5)
realname    100
username      0
gender        0
age           0
meslek       16
dtype: int64
realname     989
username    1090
gender         2
age           63
meslek        75
dtype: int64


In [17]:
users_age_gender[users_age_gender.meslek == 'CEO']

,realname,username,gender,age,meslek
69,Ebru Dorman,Ebru_Dorman,k,49,CEO
70,derya matras,derya matras,k,39,CEO
71,Damla Birol,Damlabirol,k,57,CEO
72,Tijen Akdogan,TijenAk,k,55,CEO
73,isil hasdemir,isilhasdemir,k,52,CEO
74,Özlem Dağ,dagozlem,k,49,CEO
75,Feyza Narli,FeyzaNarli,k,48,CEO
76,Zeynep Selgur,zeynepselgur_,k,58,CEO
77,Banu Guney Ariduru,BanuAriduru,k,58,CEO
78,mehmetaligoksu,Mehmet Ali Göksu,e,34,CEO


In [24]:
tweets['username1'] = tweets['username']
users_age_gender['username2'] = users_age_gender['username']

In [37]:
tweet_users = set(tweets.username1.unique())
label_users = set(users_age_gender.username2.unique())

In [40]:
print(len(tweet_users), len(label_users))

1106 1090


In [41]:
intersect = tweet_users.intersection(label_users)

1050

In [45]:
tweets[tweets['username1'] == "Çağdaş Çağlar"]

,id_str,created_at,text,entities,retweeted,username,username1
0,1144844578442350592,2019-06-29 05:46:32+00:00,RT @ekrem_imamoglu: Bu tablonun anlamı benim i...,"{'hashtags': [], 'symbols': [], 'user_mentions...",True,Çağdaş Çağlar,Çağdaş Çağlar
1,1010840786500571137,2018-06-24 11:03:18+00:00,RT @Urbarli: Dün Ankara Mitingi’nde bu video g...,"{'hashtags': [], 'symbols': [], 'user_mentions...",True,Çağdaş Çağlar,Çağdaş Çağlar
2,853511122208645120,2017-04-16 07:31:23+00:00,RT @SarkiParcalari: Mustafa Kemal Atatürk ! ht...,"{'hashtags': [], 'symbols': [], 'user_mentions...",True,Çağdaş Çağlar,Çağdaş Çağlar
3,853510789159931904,2017-04-16 07:30:04+00:00,RT @kacsaatolldu: Darıca Halk Pazarında bir te...,"{'hashtags': [{'text': 'YarınHayırÇıkacak', 'i...",True,Çağdaş Çağlar,Çağdaş Çağlar
4,827264161872633862,2017-02-02 21:15:20+00:00,@dserapunal aaaa minik kuş,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,Çağdaş Çağlar,Çağdaş Çağlar
5,822465693782306818,2017-01-20 15:27:56+00:00,@sevvallcm sensiz olmaaz,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,Çağdaş Çağlar,Çağdaş Çağlar
6,804014442337931264,2016-11-30 17:29:15+00:00,@LittleDSX yok biz 12 yiz zaten,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,Çağdaş Çağlar,Çağdaş Çağlar
7,786223152376455169,2016-10-12 15:13:01+00:00,RT @RuhuUyandir: İSTANBUL KABATAŞ ERKEK LİSESİ...,"{'hashtags': [{'text': 'RuhuUyandir', 'indices...",True,Çağdaş Çağlar,Çağdaş Çağlar
8,785934585800646656,2016-10-11 20:06:21+00:00,"RT @RuhuUyandir: BOŞ DURMADIK, SÜREKLİ ÇALIŞTI...","{'hashtags': [{'text': 'RuhuUyandır', 'indices...",True,Çağdaş Çağlar,Çağdaş Çağlar
9,785887756438364161,2016-10-11 17:00:16+00:00,@LittleDSX bebisim ben her gün ordayım yalnız...,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,Çağdaş Çağlar,Çağdaş Çağlar


In [27]:
# Merge tweet and user information
df = tweets.merge(users_age_gender, left_on='username1', right_on='username2', how='left')

# print(df.username.nunique())

# print(df.dropna(subset=['age']).username.nunique(),
# df.dropna(subset=['gender']).username.nunique())

# print(df.isna().sum())

df.dropna(subset=['age'],inplace=True)
df.dropna(subset=['gender'],inplace=True)

# print(df.isna().sum())

In [42]:
df[df.meslek == 'CEO']

,id_str,created_at,text,entities,retweeted,username_x,username1,realname,username_y,gender,age,meslek,username2
13186,1144844578442350592,2019-06-29 05:46:32+00:00,RT @ekrem_imamoglu: Bu tablonun anlamı benim i...,"{'hashtags': [], 'symbols': [], 'user_mentions...",True,Çağdaş Çağlar,Çağdaş Çağlar,bcagdascaglar,Çağdaş Çağlar,e,42.0,CEO,Çağdaş Çağlar
13187,1010840786500571137,2018-06-24 11:03:18+00:00,RT @Urbarli: Dün Ankara Mitingi’nde bu video g...,"{'hashtags': [], 'symbols': [], 'user_mentions...",True,Çağdaş Çağlar,Çağdaş Çağlar,bcagdascaglar,Çağdaş Çağlar,e,42.0,CEO,Çağdaş Çağlar
13188,853511122208645120,2017-04-16 07:31:23+00:00,RT @SarkiParcalari: Mustafa Kemal Atatürk ! ht...,"{'hashtags': [], 'symbols': [], 'user_mentions...",True,Çağdaş Çağlar,Çağdaş Çağlar,bcagdascaglar,Çağdaş Çağlar,e,42.0,CEO,Çağdaş Çağlar
13189,853510789159931904,2017-04-16 07:30:04+00:00,RT @kacsaatolldu: Darıca Halk Pazarında bir te...,"{'hashtags': [{'text': 'YarınHayırÇıkacak', 'i...",True,Çağdaş Çağlar,Çağdaş Çağlar,bcagdascaglar,Çağdaş Çağlar,e,42.0,CEO,Çağdaş Çağlar
13190,827264161872633862,2017-02-02 21:15:20+00:00,@dserapunal aaaa minik kuş,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,Çağdaş Çağlar,Çağdaş Çağlar,bcagdascaglar,Çağdaş Çağlar,e,42.0,CEO,Çağdaş Çağlar
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1445005,804014442337931264,2016-11-30 17:29:15+00:00,@LittleDSX yok biz 12 yiz zaten,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,Hande Çilingir,Hande Çilingir,HandeCilingir,Hande Çilingir,k,36.0,CEO,Hande Çilingir
1445006,786223152376455169,2016-10-12 15:13:01+00:00,RT @RuhuUyandir: İSTANBUL KABATAŞ ERKEK LİSESİ...,"{'hashtags': [{'text': 'RuhuUyandir', 'indices...",True,Hande Çilingir,Hande Çilingir,HandeCilingir,Hande Çilingir,k,36.0,CEO,Hande Çilingir
1445007,785934585800646656,2016-10-11 20:06:21+00:00,"RT @RuhuUyandir: BOŞ DURMADIK, SÜREKLİ ÇALIŞTI...","{'hashtags': [{'text': 'RuhuUyandır', 'indices...",True,Hande Çilingir,Hande Çilingir,HandeCilingir,Hande Çilingir,k,36.0,CEO,Hande Çilingir
1445008,785887756438364161,2016-10-11 17:00:16+00:00,@LittleDSX bebisim ben her gün ordayım yalnız...,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,Hande Çilingir,Hande Çilingir,HandeCilingir,Hande Çilingir,k,36.0,CEO,Hande Çilingir


In [5]:
'''
    This option is to normalize the tweets by user age
    But this option causes duplicated users since tweets are divided into years\
        if a user's tweets are spanning more than one time period
'''
df["year"] = df.created_at.apply(lambda x: x[:4]).astype(int)
df["year_relative"] = df.year.apply(lambda x: 2021 - x)
df["age_normalized"] = df.age - df.year_relative

In [6]:
from sklearn import preprocessing

# Encode gender labels
le = preprocessing.LabelEncoder()
df['gender_enc'] = le.fit_transform(df.gender.astype(str))

# Split age range into parts
categories = pd.cut(df.age,
                    bins=[0,30,40,50,60,1000],
                    labels=['0-30','31-40','41-50','51-60','61+'])
df.insert(len(df.columns),'age_group',categories)

categories_norm = pd.cut(df.age_normalized,
                    bins=[0,30,40,50,60,1000],
                    labels=['0-30','31-40','41-50','51-60','61+'])
df.insert(len(df.columns),'age_group_norm',categories_norm)

# Encode age labels
le = preprocessing.LabelEncoder()
df['age_enc'] = le.fit_transform(df.age_group.astype(str))

le = preprocessing.LabelEncoder()
df['age_enc_norm'] = le.fit_transform(df.age_group_norm.astype(str))

In [10]:
df.to_csv('tweet_data.csv',index=False)

### Celebrity Profiling Dataset

In [11]:
import pandas as pd
celeb_tweets = pd.read_json(f'celebrity_profiling/ACL-19/celebrity-profiling/hydrated/webis-celebrity-corpus-2019-hydrated.ndjson', lines=True)
celeb_info = pd.read_json(f'celebrity_profiling/ACL-19/celebrity-profiling/webis-celebrity-corpus-2019-distribution.ndjson', lines=True)

In [31]:
celeb_tweets.head()

,id,statuses_count,screen_name,lang,followers_count,name,timeline
0,21447363,11463,katyperry,NaN,108764508,KATY PERRY,[daylight savings is daytime terror for parent...
1,27260086,31385,justinbieber,NaN,114060567,Justin Bieber,[This is my brother @DevoTaylorDrums. You such...
2,813286,16296,BarackObama,NaN,130125053,Barack Obama,[We’ve done some important work since the Pari...
3,79293791,10594,rihanna,NaN,103414484,Rihanna,[the RIH 👏🏿 ISSUE 👏🏿 cop these limited edition...
4,17919972,693,taylorswift13,NaN,88959383,Taylor Swift,[November 12. Remember it. \n\n@SadieSink \n@d...


In [26]:
celeb_info.head()

,labels,id
0,"{'spouse (P26)': 'Russell Brand (Q296609)', 'g...",21447363
1,{'height (P2048)': {'unit': 'http://www.wikida...,27260086
2,{'nominated for (P1411)': ['Grammy Award for B...,813286
3,{'net worth estimate (P2218)': [{'unit': 'http...,79293791
4,{'nominated for (P1411)': 'Grammy Award for Al...,17919972


In [30]:
import json
from pprint import pprint

for idx, row in celeb_info.iterrows():
    print(row.id)
    pprint(row.labels)
    # age = row.labels["date of birth"]
    # gender = row.labels["sex or gender (P21)"]
    break

21447363
{'birth name (P1477)': 'Katheryn Elizabeth Hudson',
 'country of citizenship (P27)': 'United States of America (Q30)',
 'date of birth (P569)': '+1984-10-25T00:00:00Z',
 'discography (P358)': 'Katy Perry discography (Q1415272)',
 'ethnic group (P172)': ['English American (Q1344183)',
                         'German American (Q141817)',
                         'Irish American (Q1075293)',
                         'Portuguese American (Q3267812)'],
 'eye color (P1340)': 'blue (Q17122834)',
 'family name (P734)': ['Perry (Q2793334)', 'Hudson (Q2720681)'],
 'genre (P136)': ['pop music (Q37073)',
                  'rock music (Q11399)',
                  'pop rock (Q484641)',
                  'dance music (Q316930)',
                  'synthpop (Q1298934)'],
 'given name (P735)': ['Elizabeth (Q385468)', 'Katheryn (Q6376585)'],
 'height (P2048)': {'amount': '+68',
                    'unit': 'http://www.wikidata.org/entity/Q218593'},
 'instrument (P1303)': ['guitar (Q6607)', 'voi